## Discovery Cluster:

create a copy of your param file, e.g.:

```bash 
cd slic_matlb/slic_cdc
cp param_solvent_cdc.m test_param_solvent_cdc_1.m```

add a vector of initial guesses at the end of the for loop:

```python
addProblemCosmo(training_set{i}, pqrAll{i}, srfFile{i}, chargeDist{i}, ...
                  referenceData{i}, soluteAtomAreas{i}, soluteAtomTypes{i}, ...
                  soluteHbondData{i}, solute_VdWV{i}, solute_VdWA{i}, ...
                  solventAtomAreas{i}, solventAtomTypes{i}, ...
                  solventHbondData{i}, solvent_VdWV{i}, solvent_VdWA{i}, ...
                  atom_vols{i}, temp{i});
end

% optimization

x0_1 = [0.227 -31.964 0.315 0.501 0.047 ... %slic es
        0.480 0.899 0.262 0.002 0.360 0.841 ... %disp
        0.466 1.000 0.788 0.990 0.601 0.178 ... %disp cont.
        0.562 0.008 0.853 0.840 0.429 0.408 ... %disp cont.
        0.906 0.369 0.529 0.575 0.568 0.229 ... %hbond
        0.397 0.407 0.780 0.446 0.671 0.614 ... %hbond 
        2 ... % combinatorial z
        1]; % cavity   
x0_2 = [0.461 -17.318 0.837 0.538 0.539 ... %slic es
        0.669 0.422 0.453 0.056 0.978 0.469 ... %disp
        0.517 0.539 0.310 0.721 0.486 0.430 ... %disp cont.
        0.788 0.380 0.552 0.653 0.889 0.895 ... %disp cont.
        0.626 0.454 0.000 0.481 0.870 0.277 ... %hbond
        0.436 0.608 0.933 0.160 0.559 0.627 ... %hbond 
        3 ... % combinatorial z
        1.2]; % cavity   
x0_3 = [0.582 -48.195 0.489 0.514 0.523 ... %slic es
        0.054 0.010 0.478 0.539 0.461 0.732 ... %disp
        0.415 0.185 0.695 0.824 0.562 0.824 ... %disp cont.
        0.081 0.566 0.399 0.989 0.197 0.674 ... %disp cont.
        0.173 0.027 0.224 0.438 0.926 0.031 ... %hbond
        0.820 0.715 0.254 0.161 0.585 0.204 ... %hbond 
        1.5 ... % combinatorial z
        0.25]; % cavity   

x0 = x0_1;

...
...


% save the results
save('OptSlicCdc_1.mat', 'x', 'training_set', 'mol_list', 'ref', ...
     'calc', 'es', 'np', 'hb', 'disp', ...
     'disp_slsl', 'disp_svsl', 'disp_svsv', 'comb', 'cav', ...
     'disp_mob', 'cav_mob', 'np_mob', 'es_mob', 'np_SLIC', ...
     'rmse', 'rmse_np', 'rmse_disp', 'rmse_cav', 'rmse_es', 'rmse_eshb', ...
     'x0', 'calc0', 'es0', 'np0', 'hb0', 'disp0', 'disp_slsl0', ...
     'disp_svsl0', 'disp_svsv0', 'comb0', 'cav0', 'epsOut');```


create a file called `batch_matlab_discovery.sh`:
    
```bash
#!/bin/bash
file_name=$1
echo "#!/bin/bash" > batch_"$file_name".script
echo "#SBATCH --job-name="$file_name"" >> batch_"$file_name".script
echo "#SBATCH --output=out_%j.txt" >> batch_"$file_name".script
echo "#SBATCH --partition=long" >> batch_"$file_name".script
echo "#SBATCH --exclude=c3079" >> batch_"$file_name".script
echo "#SBATCH --nodes=1" >> batch_"$file_name".script
echo "#SBATCH --time=1-00:00:00" >> batch_"$file_name".script
echo "#SBATCH --exclusive" >> batch_"$file_name".script
echo "#SBATCH --mem=0" >> batch_"$file_name".script
echo "module load matlab/R2019a" >> batch_"$file_name".script
echo "module load anaconda3/3.7" >> batch_"$file_name".script
echo "module unload anaconda2/2018.12" >> batch_"$file_name".script
echo "matlab -nosplash -nodesktop -r \"parpool('local',12);test_param_solvent_cdc_"$file_name";test_run_solvent_cdc_"$file_name";exit\"" >> batch_"$file_name".script
/usr/bin/sbatch batch_"$file_name".script```

run

```bash
for i in {1..10}; 
do cp test_param_solvent_cdc_1.m test_param_solvent_cdc_"$i".m;
cp test_run_solvent_cdc.m test_run_solvent_cdc_"$i".m;
sed -i "s/x0 = x0_1;/x0 = x0_"$i";/g" test_param_solvent_cdc_"$i".m; 
sed -i -e "s/'OptSlicCdc_1.mat'/'OptSlicCdc_"$i".mat'/g" test_param_solvent_cdc_"$i".m; 
sed -i -e "s/'OptSlicCdc_1.mat'/'OptSlicCdc_"$i".mat'/g" test_run_solvent_cdc_"$i".m; 
sed -i -e "s/'RunSlicCdc_1.mat'/'RunSlicCdc_"$i".mat'/g" test_run_solvent_cdc_"$i".m; 
sed -i -e "s/logfileName = 'param_logfile_1';/logfileName = 'param_logfile_"$i"';/g" test_param_solvent_cdc_"$i".m; 
sed -i -e "s/logfileName = 'run_logfile_1';/logfileName = 'run_logfile_"$i"';/g" test_run_solvent_cdc_"$i".m;
./batch_matlab_discovery.sh "$i";
sleep 1m; done```